<a href="https://www.kaggle.com/code/marinabalakina/synthetic-data-generation-with-mistral-7b-instruct?scriptVersionId=166973127" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Synthetic Data Generation with Mistral 7B Instruct


In [2]:
import os
import re
import numpy as np
import pandas as pd
import csv 
import random
import time
from tqdm.notebook import tqdm
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
import sys
!{sys.executable} -m venv venv

!pip install -q -U transformers
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.6 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=121377455 sha256=93b263332e8b1ec19b7a8c5c956e463f274fe4e8b3f46b075615442b90c0433a
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash-attn


In [4]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
%%time

def generate_data(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    
    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens = 250,
            do_sample = True,
            pad_token_id = tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text



CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.3 µs


In [8]:
import pickle
# Это сырые и предобработанные отрицательные отзывы - обучающий датасет
with open('/kaggle/input/diplom/negatives.pkl', 'rb') as f:
    (raw_texts, all_neg_texts_) = pickle.load(f)

In [15]:
%%time
for review in raw_texts[:10]:
    prompt = f"Напиши ответ на отзыв '{review}'.В ответе поблагодари за отзыв, вырази понимание проблемы и сообщи о предпринимаемых мерах по ее устранению. Уложись в 3 предложения"

    response = generate_data(prompt) 
    print('Отзыв: ', review)
    print('Oтвет:', response)
    print()

Отзыв:  размер не соответствует. замок пришит наискосок. короче на 2. шел очень долго
Oтвет:  Ответ: Спасибо за Ваш отзыв. Наш команд понимает проблему, которую Вас рассказали. Мы предпринимаем следующие шаги для устранения проблемы:

1. Выбераем материал, который обеспечивает правильный размер и форму, прежде чем мы примениваем его к замоку.
2. Проводим изменения в конструкции замка, чтобы подобрать подходящую и правильную ориентацию.
3. Тестируем замок на различных у

Отзыв:  Это ужас!!! Никому не советую покупать... Одежда воняет чем то просто невыносимым... мы уже и постирали и в чистку сдали... ничего не действует!!! Я очень огорчена!
Oтвет:  1. Спасибо, что вы нашли время отметить наше устранение. Мы извиняемся за неудовлетворяющее Ваше ожидание. мы понимаем Ваших обеих проблем - яркий запах и бесполезных предотвращающих мер. мы прилагаем огромные усилия по устранению этой проблемы и разработки товарной линии, которая будет с чистотой и без дополнительных химических веществ. Мы о